# VLM Model Comparison for FADA

**Date**: October 2, 2025  
**Purpose**: Compare vision-language models for fetal ultrasound VQA  
**Hardware**: RTX 4070 (8GB VRAM)  

## Models Tested

This notebook compares 8 vision-language models tested on the FADA fetal ultrasound dataset:

1. **BLIP-2** (Baseline) - 3.4B params, 4.2GB memory
2. **FetalCLIP** (Domain-specific) - 0.4B params, 3.0GB memory
3. **SmolVLM-500M** - 0.51B params, 1.0GB memory
4. **Moondream2** - 1.93B params, 4.5GB memory
5. **BLIP-VQA-base** - 0.36B params, 1.5GB memory
6. **VILT-b32** - 0.12B params, 0.5GB memory
7. **SmolVLM-256M** (World's smallest VLM) - 0.26B params, 1.0GB memory
8. **Florence-2-large** - 0.78B params, 1.55GB memory

## Test Setup

**Question Used**: "What anatomical structures can you see in this ultrasound image?"  
**Test Images**: 5 images from 5 categories (Abodomen, Aorta, Cervical, Cervix, Femur)  
**Evaluation**: Zero-shot inference (no fine-tuning)

## Results Summary

| Model | Parameters | Memory | Speed | Fetal Context | Response Quality | Status |
|-------|-----------|--------|-------|---------------|-----------------|--------|
| **BLIP-2** | 3.4B | 4.2GB | ~5-6s | ✅ Yes | Good | ✅ Working |
| **Moondream2** | 1.93B | 4.5GB | 1.2s | ✅ Yes | Fair (generic fetal) | ✅ Working |
| **Florence-2** | 0.78B | 1.55GB | 0.2-4.5s | ✅ Yes | Fair (mixed quality) | ⚠️ Complex setup |
| **SmolVLM-500M** | 0.51B | 1.0GB | 4.5s | ❌ No | Good anatomy, wrong context | ✅ Working |
| **FetalCLIP** | 0.4B | 3.0GB | Fast | ✅ Yes | 40% accuracy | ⚠️ Category mismatch |
| **BLIP-VQA-base** | 0.36B | 1.5GB | 0.2s | ❌ No | Very short (1-2 words) | ✅ Working |
| **SmolVLM-256M** | 0.26B | 1.0GB | 5.1s | ❌ No | Detailed but generic | ✅ Working |
| **VILT-b32** | 0.12B | 0.5GB | 0.1s | ❌ No | Nonsensical (fixed vocab) | ❌ Not suitable |

## Model Details

### 1. BLIP-2 (Baseline)

**Specs**:
- Parameters: 3.4B
- Memory: 4.2GB
- Speed: ~5-6s/image

**Sample Response** (Abodomen_001.png):  
_"This ultrasound image shows fetal anatomy including the fetal abdomen with visible stomach bubble and liver...
_

**Pros**:
- Trained on medical VQA tasks
- Recognizes fetal context
- Detailed responses

**Cons**:
- Larger model (requires 4-5GB VRAM)
- Slower inference

**Verdict**: ✅ **Current best choice for FADA**

### 2. Moondream2

**Specs**:
- Parameters: 1.93B
- Memory: 4.5GB
- Speed: 1.2s/image

**Sample Responses**:
- Abodomen_001: "In this ultrasound image, you can see an embryo, fetal fetus, and possibly a placenta."
- Aorta_001: "In this ultrasound image, you can see an abdominal wall with a central line running through it."

**Pros**:
- Recognizes fetal context ("embryo", "fetus")
- Fast inference
- Optimized for edge deployment

**Cons**:
- Generic descriptions
- Similar memory usage to BLIP-2
- Less detailed than BLIP-2

**Verdict**: ⚠️ **Good alternative but not better than BLIP-2**

### 3. SmolVLM-500M

**Specs**:
- Parameters: 0.51B
- Memory: 1.0GB
- Speed: 4.5s/image

**Sample Responses**:
- Abodomen_001: "The anatomical structures visible in this ultrasound image include the uterus, cervix, and fallopian tubes."
- Aorta_001: "The image contains a human heart... with the superior vena cava (SVC) vein and the aorta... right atrium (RA) and right ventricle (RV)..."

**Pros**:
- Very efficient (1GB memory)
- Good anatomical knowledge
- Detailed descriptions

**Cons**:
- **No fetal context** - describes adult anatomy
- Identifies maternal structures instead of fetal

**Verdict**: ❌ **Not suitable - lacks domain knowledge**

### 4. FetalCLIP

**Specs**:
- Parameters: ~0.4B
- Memory: ~3.0GB
- Accuracy: 40% (zero-shot)

**Results**: 
- Tested: 15 images (5 categories)
- Correct: 6/15 (40%)
- Best: Cervix (100%), Femur (100%)
- Worst: Abdomen (0%), Aorta (0%)

**Pros**:
- Domain-specific (trained on 210K fetal ultrasounds)
- Recognizes fetal anatomy

**Cons**:
- **Category mismatch** ("Abodomen" vs "Abdomen", "Cervical" vs "Cervix")
- Classification only (no VQA)
- Dataset not publicly available

**Verdict**: ❌ **Not suitable - category alignment issues**

### 5. BLIP-VQA-base

**Specs**:
- Parameters: 0.36B
- Memory: 1.5GB
- Speed: 0.2s/image (very fast!)

**Sample Responses**:
- Abodomen_001: "torso"
- Abodomen_002: "torso"
- Abodomen_003: "stomach"
- Aorta_001: "teeth"
- Aorta_002: "teeth"

**Pros**:
- Very fast inference
- Small memory footprint
- BLIP architecture (same as BLIP-2)

**Cons**:
- **Very short responses** (1-2 words)
- Limited detail
- Generic/incorrect answers

**Verdict**: ❌ **Not suitable - insufficient detail for medical VQA**

### 6. VILT-b32

**Specs**:
- Parameters: 0.12B (smallest tested)
- Memory: 0.5GB (most efficient)
- Speed: 0.1s/image (fastest!)

**Sample Responses**:
- Abodomen_001: "scissors"
- Abodomen_002: "scissors"
- Abodomen_003: "scissors"
- Aorta_001: "tree"
- Aorta_002: "tree"

**Pros**:
- Extremely lightweight
- Very fast inference
- Minimal memory usage

**Cons**:
- **Fixed vocabulary** (VQAv2 dataset)
- Nonsensical answers for ultrasound
- Not generative (classification-based)

**Verdict**: ❌ **Not suitable - trained on natural images only**

### 7. SmolVLM-256M (World's Smallest VLM)

**Specs**:
- Parameters: 0.26B
- Memory: 1.0GB
- Speed: 5.1s/image

**Sample Responses**:
- Abodomen_001: "The image is an ultrasound image of a fetus in a mother's uterus... The fetus is oriented towards the left side..."
- Abodomen_002: "The image shows a side-by-side ultrasound of a pregnant woman, labeled as 'CH5-2 Fetal Echo'..."
- Aorta_001: "The image contains the head and neck."

**Pros**:
- Recognizes "fetal ultrasound" context
- Detailed descriptive responses
- Very efficient (1GB memory)

**Cons**:
- Generic descriptions (not anatomically specific)
- Sometimes describes image metadata instead of anatomy
- Inconsistent quality

**Verdict**: ⚠️ **Interesting but inconsistent - needs fine-tuning**

### 8. Florence-2-large

**Specs**:
- Parameters: 0.78B
- Memory: 1.55GB
- Speed: 0.2-4.5s/image

**Sample Responses**:
- Abodomen_001: "An ultrasound scan of a baby's fetus in the womb."
- Aorta_001: "a black and white photo of a tree trunk"
- Cervical_001: "a close up of a baby's ultrasound on a black background"

**Special Features**:
- Task-based prompting (<CAPTION>, <VQA>, <DETAILED_CAPTION>)
- Microsoft's vision foundation model
- Supports object detection, OCR, and grounding

**Setup Challenges**:
- Requires transformers==4.36.2 (downgrade needed)
- Flash attention dependency (bypass required)
- Separate virtual environment recommended

**Pros**:
- Recognizes ultrasound context
- Efficient memory usage (1.55GB)
- Multiple task capabilities
- Fast inference for simple captions

**Cons**:
- Compatibility issues with newer transformers
- VQA responses include location tokens
- Mixed quality (good for fetal, poor for some structures)

**Verdict**: ⚠️ **Promising but complex setup - good for specialized tasks**

## Models Requiring Special Setup (Not Tested)

### TinyGPT-V (2.8B params)
**Status**: ⚠️ SKIPPED  
**Reason**: Requires custom conda environment + Phi-2 weights + manual config  
**Effort**: 1.5-2 hours setup, 7.4GB downloads  
**Expected**: 98% of InstructBLIP performance  

### DeepSeek-VL-1.3B
**Status**: ⚠️ SKIPPED  
**Reason**: Requires git clone + pip install -e . (custom package)  
**Effort**: 10-15 minutes setup  
**Expected**: Strong reasoning on scientific tasks  

### PaliGemma-3B
**Status**: 🔒 GATED  
**Reason**: Requires HuggingFace access request  
**Effort**: Unknown approval time  
**Expected**: Google's lightweight VLM (SigLIP + Gemma)  

## Performance Comparison

### Memory Efficiency
```
VILT-b32      ▓░░░░░░░░░ 0.5GB  (most efficient)
SmolVLM-256M  ▓▓░░░░░░░░ 1.0GB
SmolVLM-500M  ▓▓░░░░░░░░ 1.0GB
BLIP-VQA      ▓▓▓░░░░░░░ 1.5GB
FetalCLIP     ▓▓▓▓▓▓░░░░ 3.0GB
BLIP-2        ▓▓▓▓▓▓▓▓░░ 4.2GB
Moondream2    ▓▓▓▓▓▓▓▓▓░ 4.5GB
```

### Inference Speed (seconds/image)
```
VILT-b32      ▓░░░░░░░░░ 0.1s   (fastest)
BLIP-VQA      ▓░░░░░░░░░ 0.2s
Moondream2    ▓▓░░░░░░░░ 1.2s
SmolVLM-500M  ▓▓▓▓▓░░░░░ 4.5s
SmolVLM-256M  ▓▓▓▓▓░░░░░ 5.1s
BLIP-2        ▓▓▓▓▓▓░░░░ 5.5s
```

### Response Quality for Medical VQA
```
BLIP-2        ▓▓▓▓▓▓▓▓▓░ 9/10  (best)
Moondream2    ▓▓▓▓▓▓░░░░ 6/10
SmolVLM-256M  ▓▓▓▓░░░░░░ 4/10
SmolVLM-500M  ▓▓▓░░░░░░░ 3/10
FetalCLIP     ▓▓▓░░░░░░░ 3/10  (category issues)
BLIP-VQA      ▓▓░░░░░░░░ 2/10  (too short)
VILT-b32      ▓░░░░░░░░░ 1/10  (nonsensical)
```

## Key Findings

### 1. Domain Knowledge is Critical
- Models without fetal/medical training produce generic or incorrect descriptions
- SmolVLM-500M (general VLM) has good anatomy knowledge but wrong context (adult vs fetal)
- FetalCLIP (domain-specific) recognizes fetal context but has category alignment issues

### 2. Model Size ≠ Medical Performance
- SmolVLM-500M (0.51B): 6.7x smaller than BLIP-2 but worse for medical VQA
- Moondream2 (1.93B): Similar size to SmolVLM but better fetal recognition
- VILT (0.12B): Smallest and fastest but completely unsuitable

### 3. Response Detail Matters
- BLIP-VQA gives 1-2 word answers ("torso", "teeth") - insufficient for medical context
- BLIP-2 provides detailed descriptions with anatomical structures
- SmolVLM-256M is verbose but often describes metadata instead of anatomy

### 4. Efficiency vs. Quality Tradeoff
- Most efficient (VILT: 0.5GB, 0.1s) → Worst quality
- Best quality (BLIP-2: 4.2GB, 5.5s) → Moderate efficiency
- Sweet spot? Moondream2 (4.5GB, 1.2s) but quality still below BLIP-2

## Recommendations

### For FADA Production (Phase 2)
**Recommended**: ✅ **BLIP-2** (current choice)  
**Rationale**:
- Best response quality for medical VQA
- Recognizes fetal anatomy context
- Fits in 8GB VRAM (RTX 4070)
- Proven baseline with training pipeline

### Alternative Considerations

**Moondream2** - Use if:
- Need faster inference (1.2s vs 5.5s)
- Can accept slightly less detailed responses
- Want edge deployment capability

**SmolVLM-256M** - Use if:
- Memory is critical constraint (<1GB)
- Willing to fine-tune on fetal ultrasound
- Need on-device deployment

### Not Recommended
- ❌ **FetalCLIP**: Category mismatch, classification-only
- ❌ **SmolVLM-500M**: No fetal context despite good anatomy
- ❌ **BLIP-VQA-base**: Too brief for medical use
- ❌ **VILT-b32**: Wrong domain (natural images)

### Future Work
1. **Fine-tune SmolVLM-256M** on fetal ultrasound → might achieve good quality + efficiency
2. **Request PaliGemma access** → Google's lightweight VLM could be promising
3. **Test with quantization** → BLIP-2 in 8-bit might reduce memory to ~2-3GB
4. **Ensemble approach** → Combine BLIP-2 (detail) + Moondream2 (speed)

## Conclusion

After testing 7 vision-language models, **BLIP-2 remains the best choice** for FADA's fetal ultrasound VQA task:

✅ **Validated**: No model tested provides better quality for medical VQA  
✅ **Research Value**: Comprehensive comparison documented for potential paper  
✅ **Decision Justified**: Systematic testing shows BLIP-2's advantages  

**Key Insight**: Domain-specific knowledge (medical/fetal) matters more than model size or inference speed for specialized medical VQA tasks.

---

**Models Tested**: 7 working + 2 fetal ultrasound
+ 3 skipped (setup complexity/gated)  
**Total Time**: ~6 hours (testing + documentation)  
**Next Steps**: Proceed with BLIP-2 fine-tuning on full FADA dataset